# Delta Lake Transaction Log

In [1]:
import pyspark
from delta import *

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [3]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-757f0266-747d-4ac7-bb56-31436ea983b8;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 381ms :: artifacts dl 30ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number|

22/11/02 11:12:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Create Delta table

In [17]:
df = spark.range(0, 3)

In [18]:
df.repartition(1).write.format("delta").save("tmp/speakers")

In [19]:
df = spark.range(8, 11)

In [20]:
df.repartition(1).write.mode("append").format("delta").save("tmp/speakers")

In [21]:
!tree tmp/speakers

tmp/speakers
├── _delta_log
│   ├── 00000000000000000000.json
│   └── 00000000000000000001.json
├── part-00000-28e1c5d4-5d4f-4091-95de-794cb333aac0-c000.snappy.parquet
└── part-00000-622f60e2-1bd4-4e46-b65d-5383ea6595bd-c000.snappy.parquet

1 directory, 4 files


In [26]:
columns = ["id"]
data = [(55,), (66,), (77,)]
rdd = spark.sparkContext.parallelize(data)
df = rdd.toDF(columns)

In [28]:
df.repartition(1).write.mode("overwrite").format("delta").save("tmp/speakers")

In [30]:
spark.read.format("delta").load("tmp/speakers").show()

+---+
| id|
+---+
| 55|
| 66|
| 77|
+---+



In [31]:
spark.read.format("delta").option("versionAsOf", "0").load("tmp/speakers").show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
+---+



In [32]:
spark.read.format("delta").option("versionAsOf", "1").load("tmp/speakers").show()

+---+
| id|
+---+
|  8|
|  9|
| 10|
|  0|
|  1|
|  2|
+---+



In [29]:
!tree tmp/speakers

tmp/speakers
├── _delta_log
│   ├── 00000000000000000000.json
│   ├── 00000000000000000001.json
│   └── 00000000000000000002.json
├── part-00000-28e1c5d4-5d4f-4091-95de-794cb333aac0-c000.snappy.parquet
├── part-00000-622f60e2-1bd4-4e46-b65d-5383ea6595bd-c000.snappy.parquet
└── part-00000-9569a078-4c6a-4f1e-8856-08c5505e738d-c000.snappy.parquet

1 directory, 6 files


In [35]:
!jq . tmp/speakers/_delta_log/00000000000000000000.json

{
  "protocol": {
    "minReaderVersion": 1,
    "minWriterVersion": 2
  }
}
{
  "metaData": {
    "id": "a6b3d7e2-6930-4a47-99a2-89e1f671f074",
    "format": {
      "provider": "parquet",
      "options": {}
    },
    "schemaString": "{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}",
    "partitionColumns": [],
    "configuration": {},
    "createdTime": 1667402134987
  }
}
{
  "add": {
    "path": "part-00000-622f60e2-1bd4-4e46-b65d-5383ea6595bd-c000.snappy.parquet",
    "partitionValues": {},
    "size": 490,
    "modificationTime": 1667402135285,
    "dataChange": true,
    "stats": "{\"numRecords\":3,\"minValues\":{\"id\":0},\"maxValues\":{\"id\":2},\"nullCount\":{\"id\":0}}"
  }
}
{
  "commitInfo": {
    "timestamp": 1667402135300,
    "operation": "WRITE",
    "operationParameters": {
      "mode": "ErrorIfExists",
      "partitionBy": "[]"
    },
    "isolationLevel": "Serializable",
    "isBlindAppend": true,
    "oper

In [36]:
!jq . tmp/speakers/_delta_log/00000000000000000000.json

{
  "protocol": {
    "minReaderVersion": 1,
    "minWriterVersion": 2
  }
}
{
  "metaData": {
    "id": "a6b3d7e2-6930-4a47-99a2-89e1f671f074",
    "format": {
      "provider": "parquet",
      "options": {}
    },
    "schemaString": "{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}",
    "partitionColumns": [],
    "configuration": {},
    "createdTime": 1667402134987
  }
}
{
  "add": {
    "path": "part-00000-622f60e2-1bd4-4e46-b65d-5383ea6595bd-c000.snappy.parquet",
    "partitionValues": {},
    "size": 490,
    "modificationTime": 1667402135285,
    "dataChange": true,
    "stats": "{\"numRecords\":3,\"minValues\":{\"id\":0},\"maxValues\":{\"id\":2},\"nullCount\":{\"id\":0}}"
  }
}
{
  "commitInfo": {
    "timestamp": 1667402135300,
    "operation": "WRITE",
    "operationParameters": {
      "mode": "ErrorIfExists",
      "partitionBy": "[]"
    },
    "isolationLevel": "Serializable",
    "isBlindAppend": true,
    "oper

In [37]:
!jq . tmp/speakers/_delta_log/00000000000000000001.json

{
  "add": {
    "path": "part-00000-28e1c5d4-5d4f-4091-95de-794cb333aac0-c000.snappy.parquet",
    "partitionValues": {},
    "size": 491,
    "modificationTime": 1667402138594,
    "dataChange": true,
    "stats": "{\"numRecords\":3,\"minValues\":{\"id\":8},\"maxValues\":{\"id\":10},\"nullCount\":{\"id\":0}}"
  }
}
{
  "commitInfo": {
    "timestamp": 1667402138600,
    "operation": "WRITE",
    "operationParameters": {
      "mode": "Append",
      "partitionBy": "[]"
    },
    "readVersion": 0,
    "isolationLevel": "Serializable",
    "isBlindAppend": true,
    "operationMetrics": {
      "numFiles": "1",
      "numOutputRows": "3",
      "numOutputBytes": "491"
    },
    "engineInfo": "Apache-Spark/3.3.0 Delta-Lake/2.1.0",
    "txnId": "033fc31e-df3d-498b-8def-e0a1c8707b88"
  }
}


In [38]:
!jq . tmp/speakers/_delta_log/00000000000000000002.json

{
  "add": {
    "path": "part-00000-9569a078-4c6a-4f1e-8856-08c5505e738d-c000.snappy.parquet",
    "partitionValues": {},
    "size": 491,
    "modificationTime": 1667403635053,
    "dataChange": true,
    "stats": "{\"numRecords\":3,\"minValues\":{\"id\":55},\"maxValues\":{\"id\":77},\"nullCount\":{\"id\":0}}"
  }
}
{
  "remove": {
    "path": "part-00000-622f60e2-1bd4-4e46-b65d-5383ea6595bd-c000.snappy.parquet",
    "deletionTimestamp": 1667403636006,
    "dataChange": true,
    "extendedFileMetadata": true,
    "partitionValues": {},
    "size": 490
  }
}
{
  "remove": {
    "path": "part-00000-28e1c5d4-5d4f-4091-95de-794cb333aac0-c000.snappy.parquet",
    "deletionTimestamp": 1667403636006,
    "dataChange": true,
    "extendedFileMetadata": true,
    "partitionValues": {},
    "size": 491
  }
}
{
  "commitInfo": {
    "timestamp": 1667403636007,
    "operation": "WRITE",
    "operationParameters": {
      "mode": "Overwrite",
      "partitionBy": "[]"
    },
    "readVersion": 1,

## Cleanup

In [16]:
!rm -rf tmp/speakers